In [4]:
import os 
import re
import cantera as ct
import rmgpy.chemkin


We need a list so we can identify the species in each merged model, as they change for each model and are even sometimes called S(bleh bleh)

In [5]:
# list_of_blends = [file for file in os.listdir('/work/westgroup/nora/Code/projects/halogens/refrigerants/blends/concatenate_equal_parts') if re.match('[A-Z0-9]+_[A-Z0-9]+', file)]
# list_of_blends            
list_of_blends = ['C2H5F_CH2F2',
 'C2H5F_CH2FCH2F',
 'C2H5F_CH2FCHF2',
 'C2H5F_CH3CHF2',
 'C2H5F_CH3F',
 'CH2F2_CH3CF3',
 'CH2FCH2F_CH2F2',
 'CH2FCH2F_CH2FCHF2',
 'CH2FCH2F_CH3CHF2',
 'CH3CF3_C2H5F',
 'CH3CF3_CH2FCH2F',
 'CH3CF3_CH2FCHF2',
 'CH3CF3_CH3CHF2',
 'CH3CHF2_CH2F2',
 'CH3CHF2_CH2FCHF2',
 'CH3F_CH3CF3',
 'CH3F_CH3CHF2']

In [6]:
#dictionary with composition of each species 

compositions = {'C2H5F':{'C': 2,'H': 5,'F': 1},
                 'C2H6':{'C': 2,'H': 6},
                 'CH2F2': {'C':1,'H':2,'F':2},
                 'CH2FCH2F':{'C':2,'H':4,'F':2},
                 'CH2FCHF2': {'C':2,'H':3,'F':3},
                 'CH3CF3': {'C':2,'H':3,'F':3},
                 'CH3CHF2': {'C':2,'H':4,'F':2},
                 'CH3F':{'C':1,'H':3,'F':1},
                 'CH4': {'C':1,'H':4},
}

smiles = {'C2H5F':'CCF',
         'C2H6':'CC',
         'CH2F2': 'FCF',
         'CH2FCH2F':'FCCF',
         'CH2FCHF2': 'FCC(F)F',
         'CH3CF3': 'CC(F)(F)F',
         'CH3CHF2': 'CC(F)F',
         'CH3F': 'CF',
         'CH4': 'C',
          'O2': '[O][O]',
}

In [7]:
#get the species ID's of each of the species in the blends

for blend in list_of_blends: 
    
    match = re.findall('([A-Z0-9]+)_([A-Z0-9]+)',blend)
    spec1 = match[0][0]
    spec2 = match[0][1]
    

    direc = f'/work/westgroup/nora/Code/projects/halogens/refrigerants/blends/blends_of_two_equal_parts_species_in_core/{blend}/chemkin'
    spcs, rxns = rmgpy.chemkin.load_chemkin_file(f'{direc}/copies/copy_chem.inp',f'{direc}/species_dictionary.txt', f'{direc}/tran.dat' )

    for species in spcs: 
        if species.smiles == smiles[spec1]: 
            print(f'{spec1} is: {species}')
        if species.smiles == smiles[spec2]: 
            print(f'{spec2} is: {species}')
    print('******************************************')

#i know some of the species in the dictionary have the same label (ex S(179)) but 
# but i've confirmed that it is still correct and that the adjacency list of the species for each model 
# is what it should be

C2H5F is: C2H5F(1)
CH2F2 is: CH2F2(34)
******************************************


C2H5F is: C2H5F(1)
CH2FCH2F is: S(179)
******************************************


C2H5F is: C2H5F(1)
CH2FCHF2 is: S(179)
******************************************


C2H5F is: C2H5F(1)
CH3CHF2 is: S(179)
******************************************


C2H5F is: C2H5F(1)
CH3F is: CH3F(34)
******************************************


...

CH3CF3 is: CH3CF3(1)
CH3CHF2 is: S(188)
******************************************


CH3CHF2 is: CH3CHF2(1)
CH2F2 is: CH2F2(40)
******************************************


CH3CHF2 is: CH3CHF2(1)
CH2FCHF2 is: S(180)
******************************************


CH3F is: CH3F(1)
CH3CF3 is: CH3CF3(81)
******************************************


CH3F is: CH3F(1)
CH3CHF2 is: C2H4F2(81)
******************************************


In [8]:
#let's save this to a dictionary that we can reference in functions below

mapping = dict()

for blend in list_of_blends: 
    
    match = re.findall('([A-Z0-9]+)_([A-Z0-9]+)',blend)
    spec1 = match[0][0]
    spec2 = match[0][1]
    
    if len(match)>2:
        print(blend)
    
    
    direc = f'/work/westgroup/nora/Code/projects/halogens/refrigerants/blends/blends_of_two_equal_parts_species_in_core/{blend}/chemkin'
    spcs, rxns = rmgpy.chemkin.load_chemkin_file(f'{direc}/copies/copy_chem.inp',f'{direc}/species_dictionary.txt', f'{direc}/tran.dat' )
   
    for_this_blend=[]
    
    for species in spcs: 
        if species.smiles == smiles[spec1]: 
            for_this_blend.append(species.to_chemkin())
        if species.smiles == smiles[spec2]: 
            for_this_blend.append(species.to_chemkin())
    
    #did this in a separate list because I want to make sure O2 is last in the list
    for species in spcs: 
        if species.smiles == smiles['O2']: 
            for_this_blend.append(species.to_chemkin())
    
 
    #add to dictionary
    mapping[blend] = for_this_blend
    

print(mapping)




{'C2H5F_CH2F2': ['C2H5F(1)', 'CH2F2(34)', 'O2(2)'], 'C2H5F_CH2FCH2F': ['C2H5F(1)', 'S(179)', 'O2(2)'], 'C2H5F_CH2FCHF2': ['C2H5F(1)', 'S(179)', 'O2(2)'], 'C2H5F_CH3CHF2': ['C2H5F(1)', 'S(179)', 'O2(2)'], 'C2H5F_CH3F': ['C2H5F(1)', 'CH3F(34)', 'O2(2)'], 'CH2F2_CH3CF3': ['CH2F2(1)', 'CH3CF3(87)', 'O2(2)'], 'CH2FCH2F_CH2F2': ['C2H4F2(1)', 'CH2F2(40)', 'O2(2)'], 'CH2FCH2F_CH2FCHF2': ['C2H4F2(1)', 'S(146)', 'O2(2)'], 'CH2FCH2F_CH3CHF2': ['C2H4F2(1)', 'S(146)', 'O2(2)'], 'CH3CF3_C2H5F': ['CH3CF3(1)', 'C2H5F(188)', 'O2(2)'], 'CH3CF3_CH2FCH2F': ['CH3CF3(1)', 'S(188)', 'O2(2)'], 'CH3CF3_CH2FCHF2': ['CH3CF3(1)', 'S(188)', 'O2(2)'], 'CH3CF3_CH3CHF2': ['CH3CF3(1)', 'S(188)', 'O2(2)'], 'CH3CHF2_CH2F2': ['CH3CHF2(1)', 'CH2F2(40)', 'O2(2)'], 'CH3CHF2_CH2FCHF2': ['CH3CHF2(1)', 'S(180)', 'O2(2)'], 'CH3F_CH3CF3': ['CH3F(1)', 'CH3CF3(81)', 'O2(2)'], 'CH3F_CH3CHF2': ['CH3F(1)', 'C2H4F2(81)', 'O2(2)']}


Make the flamespeed_calc script to go into the folder of the blends

In [16]:
import os 
import re 
import cantera as ct



def make_gas(blend_name):
    print(blend_name)
    block = \
    f'''
directory = '/work/westgroup/nora/Code/projects/halogens/refrigerants/blends/blends_of_two_equal_parts_species_in_core/{blend_name}/chemkin/copies/copy_chem.cti'
gas = ct.Solution(directory)
blend_name = '{blend_name}'

vol_frac_list = list(np.linspace(0.025, 0.25, 30))

results = dict()


#this part calculates the flame calcs for all of the vol_frac list, using the previous vol frac count as a guess for the current one
for i in  range(len(vol_frac_list)):
    try:

        string = f'****************************starting new volume fraction **************************'
        print(string)
        
        x = vol_frac_list[i]
        norm_ox = (1-x)*.21
'''
    return block

def make_vol_frac_dict(label_list,num):
    print(label_list)
    '''Makes a volume fraction dictionary, assuming there are equal parts fuel in the blend'''

    vol_frac_dict = \
    '''
        vol_frac_dict = {'''
    for label in label_list[:-1]:
        print(label)
        vol_frac_dict += f"'{label}': (x/{num}/norm_ox), "
    vol_frac_dict += f"'{label_list[-1]}':((1-x)*.21)/norm_ox, 'N2':((1-x)*0.79)/norm_ox"
    vol_frac_dict += '}'
    
    block2 = \
    '''
        gas.TPX = To, Po, vol_frac_dict
        width = 0.08
        flame = ct.FreeFlame(gas, width=width)
        flame.set_refine_criteria(ratio=3, slope=0.1, curve=0.1) 
        flame.max_time_step_count = 900
        loglevel = 1 
        try:
            if i!=0:
                d = f'data/{vol_frac_list[i-1]}.csv'
                if os.path.exists(d):  
                    arr2 = ct.SolutionArray(gas)
                    arr2.read_csv(d)
                    flame.set_initial_guess(data=arr2)
                    print(' initial guess has been set')
        except: 
            print('initial guess not set for this volume fraction')
        
        
        #"False" stops the calculation from retrying over and over
        flame.solve(loglevel=loglevel, auto=False)
        Su = flame.velocity[0]
        results[x] = Su
        sltn = flame.to_solution_array()
        df1 = sltn.to_pandas()
        df1.to_csv(f'data/{x}.csv', index=False)            
            
    except Exception as e: 
        print(f'********************passed volume fraction:{vol_frac_list[i]}, error: {e}*************************************')
        pass

vol_fracs = list(results.keys())
flame_speeds = list(results.values())


print("volume fractions are:")
print(vol_fracs)

print("flame speeds are:")
print(flame_speeds)

    
with open('blend_flamespeeds.csv', 'w+') as g:
    writers = csv.writer(g)
    writers.writerow(vol_fracs)
    writers.writerow(flame_speeds)
        
'''
    vol_frac_dict += block2
    return vol_frac_dict


def make_bash_script(label_list, time_limit, node):
    bash_script = f'''#!/bin/sh

#SBATCH --nodes=1
#SBATCH --time={time_limit}-00:00:00
#SBATCH --job-name=fc_{'_'.join(label_list[0:-1])}
#SBATCH --error=error_fc.slurm.log
#SBATCH --output=output_fc.slurm.log
##SBATCH --cpus-per-task=5
##SBATCH --mem-per-cpu=8Gb
##SBATCH --ntasks=1 
##SBATCH --array=1
#SBATCH --partition={node}

source activate cantera_env
python flame_speed_calc.py

'''
    return bash_script

In [17]:
def make_blends(list_of_blend_files, num, dictionary_of_species):
    """Proved with a list of blend files, 
    specify the number of fuel species in the blend (num), this will run flame speed calc on all of them.
    
    example: make_blend([list using os.listdir(directory)])
    """
    

    for file in list_of_blend_files: 
             
            
            
        flame_speed_calc = '''

from os import uname
import cantera as ct
import numpy as np
import pandas as pd
import os
import csv

print("Running Cantera Version: " + str(ct.__version__))

To = 298
Po = 1e5 # ct.one_atm

    '''    
            
            
        
        print(f'Now working on: {file}')

        #now make the damn thing
        flame_speed_calc += make_gas(file)
        flame_speed_calc += make_vol_frac_dict(mapping[file],num)
        bash_script = make_bash_script(mapping[file], '1', 'short')
        directory_to_put_script =f'/work/westgroup/nora/Code/projects/halogens/refrigerants/blends/blends_of_two_equal_parts_species_in_core/flamespeeds/{file}'
        with open(os.path.join(directory_to_put_script,f'flame_speed_calc.py'),'w') as f: 
            for l in flame_speed_calc: 
                f.write(l)
        with open(os.path.join(directory_to_put_script,f'flame_speed_run.sh'),'w') as f: 
            for l in bash_script: 
                f.write(l)
                
        os.makedirs(os.path.join(directory_to_put_script,'data'),exist_ok=True)
  



In [18]:
# flame_dir = '/work/westgroup/nora/Code/projects/halogens/refrigerants/blends/blends_of_two_equal_parts_species_in_core/flamespeeds/'



# list_of_blends = [file for file in os.listdir(flame_dir) if re.match('[A-Z0-9]+_[A-Z0-9]+', file)]

list_of_blends = ['C2H5F_CH2F2',
 'C2H5F_CH2FCH2F',
 'C2H5F_CH2FCHF2',
 'C2H5F_CH3CHF2',
 'C2H5F_CH3F',
 'CH2F2_CH3CF3',
 'CH2FCH2F_CH2F2',
 'CH2FCH2F_CH2FCHF2',
 'CH2FCH2F_CH3CHF2',
 'CH3CF3_C2H5F',
 'CH3CF3_CH2FCH2F',
 'CH3CF3_CH2FCHF2',
 'CH3CF3_CH3CHF2',
 'CH3CHF2_CH2F2',
 'CH3CHF2_CH2FCHF2',
 'CH3F_CH3CF3',
 'CH3F_CH3CHF2']


In [21]:
len(list_of_blends)

17

In [19]:
make_blends(list_of_blends, 2, for_this_blend)

Now working on: C2H5F_CH2F2
C2H5F_CH2F2
['C2H5F(1)', 'CH2F2(34)', 'O2(2)']
C2H5F(1)
CH2F2(34)
Now working on: C2H5F_CH2FCH2F
C2H5F_CH2FCH2F
['C2H5F(1)', 'S(179)', 'O2(2)']
C2H5F(1)
S(179)
Now working on: C2H5F_CH2FCHF2
C2H5F_CH2FCHF2
['C2H5F(1)', 'S(179)', 'O2(2)']
C2H5F(1)
S(179)
Now working on: C2H5F_CH3CHF2
C2H5F_CH3CHF2
['C2H5F(1)', 'S(179)', 'O2(2)']
C2H5F(1)
S(179)
Now working on: C2H5F_CH3F
C2H5F_CH3F
['C2H5F(1)', 'CH3F(34)', 'O2(2)']
C2H5F(1)
CH3F(34)
Now working on: CH2F2_CH3CF3
CH2F2_CH3CF3
['CH2F2(1)', 'CH3CF3(87)', 'O2(2)']
CH2F2(1)
CH3CF3(87)
Now working on: CH2FCH2F_CH2F2
CH2FCH2F_CH2F2
['C2H4F2(1)', 'CH2F2(40)', 'O2(2)']
C2H4F2(1)
CH2F2(40)
Now working on: CH2FCH2F_CH2FCHF2
CH2FCH2F_CH2FCHF2
['C2H4F2(1)', 'S(146)', 'O2(2)']
C2H4F2(1)
S(146)
Now working on: CH2FCH2F_CH3CHF2
CH2FCH2F_CH3CHF2
['C2H4F2(1)', 'S(146)', 'O2(2)']
C2H4F2(1)
S(146)
Now working on: CH3CF3_C2H5F
CH3CF3_C2H5F
['CH3CF3(1)', 'C2H5F(188)', 'O2(2)']
CH3CF3(1)
C2H5F(188)
Now working on: CH3CF3_CH2FCH2F
CH